In [1]:
import pandas as pd
import numpy as np
import pickle

from portfolios.Sharp import SharpPortfolio
from portfolios.Markov import MarkovPortfolio
from portfolios.Tobin import TobinPortfolio
from portfolios.QSharp import QSharpPortfolio

# Disable jedi autocompleter
%config Completer.use_jedi = False

In [2]:
obj = pd.read_pickle(r'result_labels.pickle')

df = pd.read_pickle(r'result_labels.pickle')
pct = pd.read_csv('ticker_data_preprocessed.csv', index_col=0)
pct = pct.drop(['sector', '2022-07-29', '2022-07-28', '2022-07-27', '2022-07-26', '2022-07-25'], axis=1)
sp500 = pd.read_csv('sp500.csv', index_col=0)['Изм. %'][5:]
bonds = pd.read_csv('bonds.csv', index_col=0)['Изм. %'][5:]

In [3]:
sp500 = sp500.apply(lambda x: float(x[:-1].replace(',', '.')) / 100)
bonds = bonds.apply(lambda x: float(x[:-1].replace(',', '.')) / 100)

# Обработка даты 

def month(dates : list):
    new_dates = []
    for d in dates:
        date = d.split()
        months = {'янв.' : '01', 'февр.': '02', 'мар.': '03', 'апр.': '04', 'мая': '05',
                  'июн.' : '06', 'июл.': '07', 'авг.': '08', 'сент.': '09', 'окт.': '10', 'нояб.': '11', 'дек.': '12'}
        date = date[0]+'-'+months[date[1]]+'-'+date[2]
        new_dates.append(date)
    return new_dates

sp500.index = month(sp500.index)

## Создаю словари для каждого метода кластеризации

In [4]:
cols = df.columns[1:]

**общий словарь**: ключ - метод кластеризации, значение - словарь(*ключ - номер кластера, значения - тикеры*)

In [5]:
all_accets = {}
for col in cols:
    accets = {}
    for i in set(df[col].unique()):
        accets[i] = list(df.loc[df[col] == i].index.values)
    all_accets[col] = accets

## Выберу из каждого кластера 2 лучшие акции по значению Шарпа и создам портфель для каждого метода кластеризации

In [6]:
def choose_best_accets(dict_accets, df_pct):
    accets_to_port = []
    for claster in dict_accets.values():
        if len(claster)==1:
            accets_to_port.append(claster[0])
        else:
            dict_with_sharp = {}
            sharp = 0
            first_sharp = -100
            second_sharp = -200
            # считаем Шарпа для каждой акции
            for ticker in claster:
                tickers_pct = df_pct.loc[df_pct.index == ticker].values
                sharp = (tickers_pct.mean()-0.03)/tickers_pct.std()
                dict_with_sharp[sharp] = ticker

            # выбираем 2 лучших акции
            for sharp in dict_with_sharp.keys():
                if first_sharp < sharp:
                    second_sharp = first_sharp
                    first_sharp = sharp
                elif first_sharp > sharp and second_sharp < sharp:
                    second_sharp = sharp
            accets_to_port.append(dict_with_sharp[first_sharp])
            accets_to_port.append(dict_with_sharp[second_sharp])
    return accets_to_port    

In [7]:
# Сохраняю расчеты в словарь: ключ - название портфеля(метод кластеризации), значение - тикеры
ports = {}
for col in cols:
    ports[col] = choose_best_accets(all_accets[col], pct)

<ipython-input-6-3202b9dbee22>:14: RuntimeWarning: Mean of empty slice.
  sharp = (tickers_pct.mean()-0.03)/tickers_pct.std()
D:\DS_IT\Anaconda\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\DS_IT\Anaconda\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
D:\DS_IT\Anaconda\lib\site-packages\numpy\core\_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
D:\DS_IT\Anaconda\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Найду лучшие методы кластеризации исходя из доходности

Веса в портфеле определяются значением коэффициента Шарпа

In [8]:
bonds.index = pd.to_datetime(bonds.index)
sp500.index = pd.to_datetime(sp500.index)

# Настройка размерности

sp500 = sp500.drop(list(set(sp500.index) - set(bonds.index)))
bonds = bonds.drop(list(set(bonds.index) - set(sp500.index)))

In [9]:
pct_monthly = {}
sp500_monthly = {}
bonds_monthly = {}

pct = pct.T[:-1]
pct.index = pd.to_datetime(pct.index)

In [10]:
# Настройка размерности

pct = pct.drop(list(set(pct.index) - set(bonds.index)))
bonds = bonds.drop(list(set(bonds.index) - set(pct.index)))
sp500 = sp500.drop(list(set(sp500.index) - set(pct.index)))

In [11]:
dates = list((sp500.reset_index().groupby(pd.Grouper(key="index", freq="M")).std()).index)

In [12]:
last_date = dates[0]
pct_monthly[last_date] = pct[(pct.index<=last_date)]
sp500_monthly[last_date] = sp500[(sp500.index<=last_date)]
bonds_monthly[last_date] = bonds[(bonds.index<=last_date)]
for date in dates[1:]:
    pct_monthly[date] = pct[((pct.index>last_date)&(pct.index<=date))]
    sp500_monthly[date] = sp500[((sp500.index>last_date)&(sp500.index<=date))]
    bonds_monthly[date] = bonds[((bonds.index>last_date)&(bonds.index<=date))]
    last_date=date

Для каждого портфеля создаю словарь с доходностями

In [13]:
def create_dict(dates, pct_monthly, tickers):
    d = {}
    for date in dates:
        d[date] = pct_monthly[date][tickers]
    return d

In [14]:
port_names = list(ports.keys())
portfolios = {}
for port in port_names:
    portfolios[port] = create_dict(dates, pct_monthly, ports[port])

В переменной portfolios хранятся словари(каждый метод кластеризации) с доходностями своих тикеров

### Расчет портфелей

1) Выбираем метод кластеризацци
2) Считаем для него оптимизацию

In [15]:
def calc_ports(test, riskless_ret, index, mu, sigma):
    sharp_values, markov_values, tobin_values, traynor_values, qsharp_values = [], [], [], [], []
    # Расчет Шарпа
    sharp = SharpPortfolio(mu, sigma, riskless_ret.mean())
    sh_values = test @ sharp.fit()
    for value in sh_values:
        sharp_values.append(value)

    # Расчет Марковица
    markov = MarkovPortfolio(mu, sigma, index)
    m_values = test @ markov.fit()
    for value in m_values:
        markov_values.append(value)

    # Расчет Тобина

    tobin = TobinPortfolio(mu, sigma, riskless_ret.mean(), index.std())
    #
    tobin.fit()
    #
    tob_values = test @ tobin.fit()
    for value in tob_values:
        tobin_values.append(value)


    df_all_ports = pd.DataFrame({'date':test.index,
                                 'sharp':sharp_values,
                                 'markov':markov_values,
                                 'tobin':tobin_values})
    return df_all_ports

In [16]:
with_god = {}
for port in list(portfolios.keys()):
    klaster = portfolios[port]
    test = klaster[dates[0]]
    riskless_ret = bonds_monthly[dates[0]]
    index = sp500_monthly[dates[0]]
    mu = test.mean().values
    sigma = test.cov().values

    with_god[port] = calc_ports(test, riskless_ret, index, mu, sigma)
    for date in dates:
        test = klaster[date]
        riskless_ret = bonds_monthly[date]
        index = sp500_monthly[date]
        mu = test.mean().values
        sigma = test.cov().values
        layer = calc_ports(test, riskless_ret, index, mu, sigma)
        with_god[port] = pd.concat([with_god[port], layer])

-0.28383377657283104 0.0994018349506062
-0.28383377657283104 0.0994018349506062
-0.46522853339043424 -0.14150455945517842
-0.15083202883973254 -0.0034212318466746204
-0.1388560274425439 0.30124560650741333
0.5156661617755177 0.38954057149917404
0.09583582945088213 0.05992721108733172
-0.26293409162349 0.04476488034627376
0.22186399359934997 0.2964507535014571
-0.17080787861090632 0.062286190965803116
-0.3119336496496764 -0.272413780566504
0.19003058837156303 0.194331874525548
-0.2145002153807543 -0.2868097121601288
0.7772374654303432 0.5264945652060085
0.39315728630297786 0.07745840905392956
0.526716745264349 -0.237673713000717
0.2178352275557806 0.1794274968050965
-0.0025390628383126852 -0.2833856027187097
0.6992975699025024 0.22067635942500602
0.046371475239329045 0.047548613814738475
-0.03973701970383061 -0.022761600559737038
0.3408035988424436 0.16401082093505778
0.5494758558896634 0.10911749535526492
0.09236862985657886 0.2815230305673494
0.2523152853148276 0.501934269613846
0.223

-0.5414675033363902 0.028750898373436622
1.6813245231936702 0.21380580164757076
1.1014514120148267 0.2549504832164811
-2.320440007942166 0.06361137798869337
-0.17812656348437303 -0.16369902425155045
0.6653519598606131 0.06508898985477678
-1.2770708002550666 -0.08919107436122871
-1.5246429547267935 0.41741814363687046
-0.382633184065753 -0.17749035078719452
-0.8142245981663832 0.046884016247367086
-1.4091317597012172 -0.019975462768158096
-0.2847426609631815 -0.047670468204221314
-1.1607123713630043 0.15972292434250132
-0.30861168240419956 -0.10774398673486298
-0.016444122453504635 0.19165289503538543
-0.535093206955851 -0.14334721022000668
0.45687348105894704 0.5969105454165031
-0.29113284883250784 0.010795432178781289
-0.29113284883250784 0.010795432178781289
-0.33178347785499646 -0.09090765468741166
-0.24096992888057742 -0.09828542708702234
-0.21882195095343698 0.10963262055929486
0.1911484490481699 0.12674562538071898
0.03848052116057058 0.04778468874287502
-0.21381656068270802 0.10

0.259262029554769 0.08520714952031481
0.6301243193202991 0.2647493292953524
0.07547831990704794 0.3232738888205448
0.34377274031525085 0.5957583700362183
0.4259958428573728 0.08721863214739357
1.3139381555405911 0.010159874814460184
0.22133156923175773 -0.14123897911487482
1.1900876868727417 0.43902734895705053
0.15586802164974745 0.6231802714525072
0.30432981385395513 0.11457054527196782
0.7240519432027128 0.2063960139156881
-0.13492790145926825 0.8699458359415826
0.8002359027305964 0.058834295907368175
0.27108392046888846 -0.021101043777746173
2.1417476373033586 0.6982281868834517
-0.13859172468963288 0.6713272381661007
0.5413163531954827 -0.034692765830819025
-0.8045246952874185 0.05527995786280033
-0.5162757675213159 -0.024142392968219125
1.8227290127812596 0.21918400025043785
0.840544630565303 0.07327008984476317
-3.716920973584356 0.692440418592239
0.03556373859190993 0.04596848412930058
1.1990611861791944 0.2657893000428864
-1.9303485338364292 -0.1530610694529286
-1.408040940562

-0.20933882225236672 -0.06887301485803925
-0.061637229241668784 0.32973475410236164
0.2871535830416196 0.20569925565022348
0.09195496604851836 0.10095274141173591
-0.24123023984937159 0.11637009851945701
0.2174111192251948 0.2796901620256873
-0.015113339155643672 0.15998675862542408
-0.35552281284751186 -0.317634718487594
0.1230983031163908 0.1265070074080744
-0.21815945779825013 -0.2711982173848493
0.6439725293090534 0.44878278125966636
0.38749352690695865 0.21453189269559633
0.44531878692832905 -0.1569248788352045
0.12217669046632579 0.1014867178008001
-0.06488673543129403 -0.263672649863099
0.49992041156503886 0.11173485003704062
0.04760061387541274 0.051731172585741086
-0.1666626715721748 -0.15256777388377463
0.27831411675191053 0.12323124281503822
0.35000672912153297 0.043825674607467335
0.2180596408744167 0.3791622353538826
0.25493344727429973 0.4307021333334752
0.27586281235871707 -0.0197445077177145
0.960339924617496 -0.20306515520224702
0.05869976315310202 -0.21545280807460185

0.8681198292298334 0.03985569844445296
0.830377070004509 0.24725804973718546
-1.2469237401717381 0.35841991188523153
-0.14765196285872914 -0.13620091603475373
0.46251949322574465 0.04728253359078611
-0.7855771855135356 0.12587431157870968
-0.7885373756284936 0.48349631404002913
-0.3035075074306875 -0.12387454136449784
-0.7482627477115031 -0.10922185220460864


D:\DS_IT\Anaconda\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


-1.0145013739368005 0.04813715560403895
-0.20623175293002247 -0.02672497159296596
-0.9982031833619761 0.21338771981306512
-0.2196550861291297 -0.06627133077704772
0.08089277571862807 0.2569967003583414
-0.3984504568956293 -0.07825264278015209
0.5220232013554674 0.633328884906797
0.005290095663207424 0.42047991933092255
0.005290095663207424 0.42047991933092255
-0.2979640435048121 -0.03158798924064334
-0.24852883024760627 -0.12325934333789523
-0.04208058789518134 0.26269777781070397
0.5081798858267222 0.42367062036145103
0.13343178850701112 0.1429193093054237
-0.29134589015775425 0.025787767417975647
-0.0037444049454197744 0.07497111618103883
-0.21400165587305925 -0.04535693544566278
-0.27201788972024393 -0.23552746580538678
0.19587904094348627 0.19784770357966042
-0.1381779591514726 -0.19087098561167068
0.7629756993066467 0.5042565897957558
0.5938007355012446 0.39568081057555854
0.6899446473437334 -0.1129216040256203
0.3005986286479876 0.2428548984362251
0.0017939883031747297 -0.2204139

In [17]:
df_res_sharp = pd.DataFrame([])
df_res_markov = pd.DataFrame([])
df_res_tobin = pd.DataFrame([])
columns = []

for i, (k, v) in enumerate(with_god.items()):
    col_names = v.columns
    df_loc_sharp = pd.DataFrame(data=v[col_names[1]].values, index=v[col_names[0]].values, columns = [k])
    df_res_sharp = pd.concat([df_res_sharp, df_loc_sharp], axis=1)
    df_loc_markov = pd.DataFrame(data=v[col_names[2]].values, index=v[col_names[0]].values, columns = [k])
    df_res_markov = pd.concat([df_res_markov, df_loc_markov], axis=1)
    df_loc_tobin = pd.DataFrame(data=v[col_names[3]].values, index=v[col_names[0]].values, columns = [k])
    df_res_tobin = pd.concat([df_res_tobin, df_loc_tobin], axis=1)
    columns.append(columns)

In [18]:
df_res_sharp['sp500']=sp500
df_res_markov['sp500']=sp500
df_res_tobin['sp500']=sp500

df_res_sharp.to_csv('df_res_sharp.csv')
df_res_markov.to_csv('df_res_markov.csv')
df_res_tobin.to_csv('df_res_tobin.csv')

In [19]:
df_real_sharp = (df_res_sharp + 1).cumprod()
df_real_markov = (df_res_markov + 1).cumprod()
df_real_tobin = (df_res_tobin + 1).cumprod()

df_real_sharp.to_csv('df_real_sharp.csv')
df_real_markov.to_csv('df_real_markov.csv')
df_real_tobin.to_csv('df_real_tobin.csv')

## Метрики Sharp

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates as mdates

from scipy.stats import norm

import functools

## Поиск метрик

In [21]:
df_all_ports_pct = pd.read_csv('df_res_sharp.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [22]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей
#dates = ret_ports.index

## Безрисковый актив

In [23]:
df_b = pd.read_csv(r'bonds.csv', decimal=",")
df_bond = pd.DataFrame()

# Найти среднюю цену за месяц и поделить её на 100
df_bond['date'] = df_b['Дата']
df_bond['цена'] = df_b['Цена']/100
df_bond.date = pd.to_datetime(df_bond.date)
df_bond.set_index('date', inplace = True)

df_bond = df_bond.mean()

## Риск портфеля

In [24]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [25]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [26]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса IMOEX
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta = beta.reset_index(drop=True)
beta_values = beta.sp500.values

In [27]:
k = 17

original = []
KMeans_TableFeatures = []
AgglomerativeClustering_TableFeatures = []
GaussianMixture_TableFeatures = []
AffinityPropagation_TableFeatures = []
KMeans_fourie1_Fourier = [] 
AgglomerativeClustering_fourie1_Fourier = []
GaussianMixture_fourie1_Fourier = []
KMeans_fourie2_Fourier = []
AgglomerativeClustering_fourie2_Fourier = []
GaussianMixture_fourie2_Fourier = [] 
KMeans_ARIMA = []
AgglomerativeClustering_ARIMA = [] 
GaussianMixture_ARIMA = []
AffinityPropagation_ARIMA = [] 
TimeSeriesKMeans_TimeSeries = []
KShape_TimeSeries = []
sp500 = []

while k<=(len(beta_values)-1):
    original.append(beta_values[k-17])
    KMeans_TableFeatures.append(beta_values[k-16])
    AgglomerativeClustering_TableFeatures.append(beta_values[k-15])
    GaussianMixture_TableFeatures.append(beta_values[k-14])
    AffinityPropagation_TableFeatures.append(beta_values[k-13])
    KMeans_fourie1_Fourier.append(beta_values[k-12])
    AgglomerativeClustering_fourie1_Fourier.append(beta_values[k-11])
    GaussianMixture_fourie1_Fourier.append(beta_values[k-10])
    KMeans_fourie2_Fourier.append(beta_values[k-9])
    AgglomerativeClustering_fourie2_Fourier.append(beta_values[k-8])
    GaussianMixture_fourie2_Fourier.append(beta_values[k-7])
    KMeans_ARIMA.append(beta_values[k-6])
    AgglomerativeClustering_ARIMA.append(beta_values[k-5])
    GaussianMixture_ARIMA.append(beta_values[k-4])
    AffinityPropagation_ARIMA.append(beta_values[k-3])
    TimeSeriesKMeans_TimeSeries.append(beta_values[k-2])
    KShape_TimeSeries.append(beta_values[k-1])
    sp500.append(beta_values[k])
    k+=18
    
beta_ports = pd.DataFrame({'original':original,
                           'KMeans_TableFeatures':KMeans_TableFeatures,
                           'AgglomerativeClustering_TableFeatures':AgglomerativeClustering_TableFeatures,
                           'GaussianMixture_TableFeatures':GaussianMixture_TableFeatures,
                           'AffinityPropagation_TableFeatures':AffinityPropagation_TableFeatures,
                           'KMeans_fourie1_Fourier': KMeans_fourie1_Fourier, 
                           'AgglomerativeClustering_fourie1_Fourier':AgglomerativeClustering_fourie1_Fourier,
                           'GaussianMixture_fourie1_Fourier':GaussianMixture_fourie1_Fourier,
                           'KMeans_fourie2_Fourier':KMeans_fourie2_Fourier,
                           'AgglomerativeClustering_fourie2_Fourier':AgglomerativeClustering_fourie2_Fourier,
                           'GaussianMixture_fourie2_Fourier':GaussianMixture_fourie2_Fourier,
                           'KMeans_ARIMA':KMeans_ARIMA,
                           'AgglomerativeClustering_ARIMA':AgglomerativeClustering_ARIMA,
                           'GaussianMixture_ARIMA':GaussianMixture_ARIMA,
                           'AffinityPropagation_ARIMA':AffinityPropagation_ARIMA,
                           'TimeSeriesKMeans_TimeSeries':TimeSeriesKMeans_TimeSeries,
                           'KShape_TimeSeries':KShape_TimeSeries, 
                           'sp500':sp500
                        }, index = ['Beta'])

## Коэффициент VaR

In [28]:
#средний pct за месяц, стандартное отклонение за месяц

In [29]:
#percentile = df_all_ports_pct.reset_index().groupby(pd.Grouper(key="date", freq="M")).quantile([0.25])
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [30]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [31]:
var=[]
for i in range (len(std_for_var)):
    var.append(l_pr_values[i]*(1+norm.ppf(0.001, mean_for_var[i], std_for_var[i]))/l_pr_values[i]-1)

In [32]:
var = pd.DataFrame({'original':var[0], 
                    'KMeans_TableFeatures':var[1],
                    'AgglomerativeClustering_TableFeatures':var[2],
                    'GaussianMixture_TableFeatures':var[3],
                    'AffinityPropagation_TableFeatures':var[4],
                    'KMeans_fourie1_Fourier':var[5], 
                    'AgglomerativeClustering_fourie1_Fourier':var[6],
                    'GaussianMixture_fourie1_Fourier':var[7],
                    'KMeans_fourie2_Fourier':var[8],
                    'AgglomerativeClustering_fourie2_Fourier':var[9],
                    'GaussianMixture_fourie2_Fourier':var[10],
                    'KMeans_ARIMA':var[11],
                    'AgglomerativeClustering_ARIMA':var[12],
                    'GaussianMixture_ARIMA':var[13],
                    'AffinityPropagation_ARIMA':var[14],
                    'TimeSeriesKMeans_TimeSeries':var[15],
                    'KShape_TimeSeries':var[16],
                    'sp500':var[17]}, index=['VaR'])

## Коэффициент Шарпа

In [33]:
std_ports = df_all_ports_pct.std()

# Sharp = (ret_port - ret_risk_free_asset) / std(ret_port)
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [34]:
def find_max_drawdown(prices):

    max_price = prices.iloc[0]
    curr_drawdown = 0
    max_drawdown = 0
    curr_left = 0
    left = 0
    right = 0
    for i in range(0, len(prices)):
        curr_drawdown = (prices.iloc[i] / max_price - 1) * 100
        # print(type(curr_drawdown))
        if curr_drawdown < max_drawdown:
            max_drawdown = curr_drawdown
            left = curr_left
            right = i
        if prices.iloc[i] > max_price:
            max_price = prices.iloc[i]
            curr_left = i
    return max_drawdown

In [35]:
def find_max_recovery(prices):
    
    def calc_growth(prices):

        growth = []
        past_p = 0
        for p in prices:
            if past_p:
                growth.append(p - past_p)
            past_p = p
        return growth

    growth = calc_growth(prices)
    s = 0
    left = 0
    right = 0
    curr_left = 0
    max_recovery = 0
    for i in range(0, len(growth)):
        if not s:
            curr_left = i
        s += growth[i]
        if s > 0:
            s = 0
            if max_recovery < (i - curr_left):
                max_recovery = i - curr_left
                left = curr_left
                right = i

    return max_recovery

In [36]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [37]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [38]:
all_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(all_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(all_metrics.columns), axis=1, inplace=True)

all_metrics = all_metrics.append(rec)
all_metrics = all_metrics.append(drawd)

all_metrics = all_metrics.T
all_metrics['Method'] = ['Sharp']*len(all_metrics)

## Метрики Markov

## Поиск метрик

In [39]:
df_all_ports_pct = pd.read_csv('df_res_markov.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [40]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Безрисковый актив

In [41]:
df_b = pd.read_csv(r'bonds.csv', decimal=",")
df_bond = pd.DataFrame()

# Найти среднюю цену за месяц и поделить её на 100
df_bond['date'] = df_b['Дата']
df_bond['цена'] = df_b['Цена']/100
df_bond.date = pd.to_datetime(df_bond.date)
df_bond.set_index('date', inplace = True)

df_bond = df_bond.mean()

## Риск портфеля

In [42]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [43]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [44]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса IMOEX
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta = beta.reset_index(drop=True)
beta_values = beta.sp500.values

In [45]:
k = 17

original = []
KMeans_TableFeatures = []
AgglomerativeClustering_TableFeatures = []
GaussianMixture_TableFeatures = []
AffinityPropagation_TableFeatures = []
KMeans_fourie1_Fourier = [] 
AgglomerativeClustering_fourie1_Fourier = []
GaussianMixture_fourie1_Fourier = []
KMeans_fourie2_Fourier = []
AgglomerativeClustering_fourie2_Fourier = []
GaussianMixture_fourie2_Fourier = [] 
KMeans_ARIMA = []
AgglomerativeClustering_ARIMA = [] 
GaussianMixture_ARIMA = []
AffinityPropagation_ARIMA = [] 
TimeSeriesKMeans_TimeSeries = []
KShape_TimeSeries = []
sp500 = []

while k<=(len(beta_values)-1):
    original.append(beta_values[k-17])
    KMeans_TableFeatures.append(beta_values[k-16])
    AgglomerativeClustering_TableFeatures.append(beta_values[k-15])
    GaussianMixture_TableFeatures.append(beta_values[k-14])
    AffinityPropagation_TableFeatures.append(beta_values[k-13])
    KMeans_fourie1_Fourier.append(beta_values[k-12])
    AgglomerativeClustering_fourie1_Fourier.append(beta_values[k-11])
    GaussianMixture_fourie1_Fourier.append(beta_values[k-10])
    KMeans_fourie2_Fourier.append(beta_values[k-9])
    AgglomerativeClustering_fourie2_Fourier.append(beta_values[k-8])
    GaussianMixture_fourie2_Fourier.append(beta_values[k-7])
    KMeans_ARIMA.append(beta_values[k-6])
    AgglomerativeClustering_ARIMA.append(beta_values[k-5])
    GaussianMixture_ARIMA.append(beta_values[k-4])
    AffinityPropagation_ARIMA.append(beta_values[k-3])
    TimeSeriesKMeans_TimeSeries.append(beta_values[k-2])
    KShape_TimeSeries.append(beta_values[k-1])
    sp500.append(beta_values[k])
    k+=18
    
beta_ports = pd.DataFrame({'original':original,
                           'KMeans_TableFeatures':KMeans_TableFeatures,
                           'AgglomerativeClustering_TableFeatures':AgglomerativeClustering_TableFeatures,
                           'GaussianMixture_TableFeatures':GaussianMixture_TableFeatures,
                           'AffinityPropagation_TableFeatures':AffinityPropagation_TableFeatures,
                           'KMeans_fourie1_Fourier': KMeans_fourie1_Fourier, 
                           'AgglomerativeClustering_fourie1_Fourier':AgglomerativeClustering_fourie1_Fourier,
                           'GaussianMixture_fourie1_Fourier':GaussianMixture_fourie1_Fourier,
                           'KMeans_fourie2_Fourier':KMeans_fourie2_Fourier,
                           'AgglomerativeClustering_fourie2_Fourier':AgglomerativeClustering_fourie2_Fourier,
                           'GaussianMixture_fourie2_Fourier':GaussianMixture_fourie2_Fourier,
                           'KMeans_ARIMA':KMeans_ARIMA,
                           'AgglomerativeClustering_ARIMA':AgglomerativeClustering_ARIMA,
                           'GaussianMixture_ARIMA':GaussianMixture_ARIMA,
                           'AffinityPropagation_ARIMA':AffinityPropagation_ARIMA,
                           'TimeSeriesKMeans_TimeSeries':TimeSeriesKMeans_TimeSeries,
                           'KShape_TimeSeries':KShape_TimeSeries, 
                           'sp500':sp500
                        }, index = ['Beta'])

## Коэффициент VaR

In [46]:
#средний pct за месяц, стандартное отклонение за месяц

In [47]:
#percentile = df_all_ports_pct.reset_index().groupby(pd.Grouper(key="date", freq="M")).quantile([0.25])
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [48]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [49]:
var=[]
for i in range (len(std_for_var)):
    var.append(l_pr_values[i]*(1+norm.ppf(0.001, mean_for_var[i], std_for_var[i]))/l_pr_values[i]-1)

In [50]:
var = pd.DataFrame({'original':var[0], 
                    'KMeans_TableFeatures':var[1],
                    'AgglomerativeClustering_TableFeatures':var[2],
                    'GaussianMixture_TableFeatures':var[3],
                    'AffinityPropagation_TableFeatures':var[4],
                    'KMeans_fourie1_Fourier':var[5], 
                    'AgglomerativeClustering_fourie1_Fourier':var[6],
                    'GaussianMixture_fourie1_Fourier':var[7],
                    'KMeans_fourie2_Fourier':var[8],
                    'AgglomerativeClustering_fourie2_Fourier':var[9],
                    'GaussianMixture_fourie2_Fourier':var[10],
                    'KMeans_ARIMA':var[11],
                    'AgglomerativeClustering_ARIMA':var[12],
                    'GaussianMixture_ARIMA':var[13],
                    'AffinityPropagation_ARIMA':var[14],
                    'TimeSeriesKMeans_TimeSeries':var[15],
                    'KShape_TimeSeries':var[16],
                    'sp500':var[17]}, index=['VaR'])

## Коэффициент Шарпа

In [51]:
std_ports = df_all_ports_pct.std()
# Sharp = (ret_port - ret_risk_free_asset) / std(ret_port)
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [52]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [53]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [54]:
markov_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(markov_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(markov_metrics.columns), axis=1, inplace=True)

markov_metrics = markov_metrics.append(rec)
markov_metrics = markov_metrics.append(drawd)

markov_metrics = markov_metrics.T
markov_metrics['Method'] = ['Markov']*len(all_metrics)

In [55]:
all_metrics = pd.concat([all_metrics, markov_metrics])

## Метрики Tobin

## Поиск метрик

In [56]:
df_all_ports_pct = pd.read_csv('df_res_tobin.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [57]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Безрисковый актив

In [58]:
df_b = pd.read_csv(r'bonds.csv', decimal=",")
df_bond = pd.DataFrame()

# Найти среднюю цену за месяц и поделить её на 100
df_bond['date'] = df_b['Дата']
df_bond['цена'] = df_b['Цена']/100
df_bond.date = pd.to_datetime(df_bond.date)
df_bond.set_index('date', inplace = True)

In [59]:
df_bond = df_bond.mean()

## Риск портфеля

In [60]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [61]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [62]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса IMOEX
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta = beta.reset_index(drop=True)
beta_values = beta.sp500.values

In [63]:
k = 17

original = []
KMeans_TableFeatures = []
AgglomerativeClustering_TableFeatures = []
GaussianMixture_TableFeatures = []
AffinityPropagation_TableFeatures = []
KMeans_fourie1_Fourier = [] 
AgglomerativeClustering_fourie1_Fourier = []
GaussianMixture_fourie1_Fourier = []
KMeans_fourie2_Fourier = []
AgglomerativeClustering_fourie2_Fourier = []
GaussianMixture_fourie2_Fourier = [] 
KMeans_ARIMA = []
AgglomerativeClustering_ARIMA = [] 
GaussianMixture_ARIMA = []
AffinityPropagation_ARIMA = [] 
TimeSeriesKMeans_TimeSeries = []
KShape_TimeSeries = []
sp500 = []

while k<=(len(beta_values)-1):
    original.append(beta_values[k-17])
    KMeans_TableFeatures.append(beta_values[k-16])
    AgglomerativeClustering_TableFeatures.append(beta_values[k-15])
    GaussianMixture_TableFeatures.append(beta_values[k-14])
    AffinityPropagation_TableFeatures.append(beta_values[k-13])
    KMeans_fourie1_Fourier.append(beta_values[k-12])
    AgglomerativeClustering_fourie1_Fourier.append(beta_values[k-11])
    GaussianMixture_fourie1_Fourier.append(beta_values[k-10])
    KMeans_fourie2_Fourier.append(beta_values[k-9])
    AgglomerativeClustering_fourie2_Fourier.append(beta_values[k-8])
    GaussianMixture_fourie2_Fourier.append(beta_values[k-7])
    KMeans_ARIMA.append(beta_values[k-6])
    AgglomerativeClustering_ARIMA.append(beta_values[k-5])
    GaussianMixture_ARIMA.append(beta_values[k-4])
    AffinityPropagation_ARIMA.append(beta_values[k-3])
    TimeSeriesKMeans_TimeSeries.append(beta_values[k-2])
    KShape_TimeSeries.append(beta_values[k-1])
    sp500.append(beta_values[k])
    k+=18
    
beta_ports = pd.DataFrame({'original':original,
                           'KMeans_TableFeatures':KMeans_TableFeatures,
                           'AgglomerativeClustering_TableFeatures':AgglomerativeClustering_TableFeatures,
                           'GaussianMixture_TableFeatures':GaussianMixture_TableFeatures,
                           'AffinityPropagation_TableFeatures':AffinityPropagation_TableFeatures,
                           'KMeans_fourie1_Fourier': KMeans_fourie1_Fourier, 
                           'AgglomerativeClustering_fourie1_Fourier':AgglomerativeClustering_fourie1_Fourier,
                           'GaussianMixture_fourie1_Fourier':GaussianMixture_fourie1_Fourier,
                           'KMeans_fourie2_Fourier':KMeans_fourie2_Fourier,
                           'AgglomerativeClustering_fourie2_Fourier':AgglomerativeClustering_fourie2_Fourier,
                           'GaussianMixture_fourie2_Fourier':GaussianMixture_fourie2_Fourier,
                           'KMeans_ARIMA':KMeans_ARIMA,
                           'AgglomerativeClustering_ARIMA':AgglomerativeClustering_ARIMA,
                           'GaussianMixture_ARIMA':GaussianMixture_ARIMA,
                           'AffinityPropagation_ARIMA':AffinityPropagation_ARIMA,
                           'TimeSeriesKMeans_TimeSeries':TimeSeriesKMeans_TimeSeries,
                           'KShape_TimeSeries':KShape_TimeSeries, 
                           'sp500':sp500
                        }, index = ['Beta'])

## Коэффициент VaR

In [64]:
#средний pct за месяц, стандартное отклонение за месяц

In [65]:
#percentile = df_all_ports_pct.reset_index().groupby(pd.Grouper(key="date", freq="M")).quantile([0.25])
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [66]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [67]:
var=[]
for i in range (len(std_for_var)):
    var.append(l_pr_values[i]*(1+norm.ppf(0.001, mean_for_var[i], std_for_var[i]))/l_pr_values[i]-1)

In [68]:
var = pd.DataFrame({'original':var[0], 
                    'KMeans_TableFeatures':var[1],
                    'AgglomerativeClustering_TableFeatures':var[2],
                    'GaussianMixture_TableFeatures':var[3],
                    'AffinityPropagation_TableFeatures':var[4],
                    'KMeans_fourie1_Fourier':var[5], 
                    'AgglomerativeClustering_fourie1_Fourier':var[6],
                    'GaussianMixture_fourie1_Fourier':var[7],
                    'KMeans_fourie2_Fourier':var[8],
                    'AgglomerativeClustering_fourie2_Fourier':var[9],
                    'GaussianMixture_fourie2_Fourier':var[10],
                    'KMeans_ARIMA':var[11],
                    'AgglomerativeClustering_ARIMA':var[12],
                    'GaussianMixture_ARIMA':var[13],
                    'AffinityPropagation_ARIMA':var[14],
                    'TimeSeriesKMeans_TimeSeries':var[15],
                    'KShape_TimeSeries':var[16],
                    'sp500':var[17]}, index=['VaR'])

## Коэффициент Шарпа

In [69]:
std_ports = df_all_ports_pct.std()
# Sharp = (ret_port - ret_risk_free_asset) / std(ret_port)
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [70]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [71]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [72]:
tobin_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(tobin_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(tobin_metrics.columns), axis=1, inplace=True)

tobin_metrics = tobin_metrics.append(rec)
tobin_metrics = tobin_metrics.append(drawd)

tobin_metrics = tobin_metrics.T
tobin_metrics['Method'] = ['Tobin']*len(tobin_metrics)

In [73]:
all_metrics = pd.concat([all_metrics, tobin_metrics])

## Сохранение результата

In [74]:
all_metrics.to_excel('metrics.xlsx')